In [1]:
# %pip install nbformat

In [2]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [3]:
%run ml_project_normalized.ipynb

Note: you may need to restart the kernel to use updated packages.


/u/nneoma/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
Done running ml_project.ipynb.


In [4]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
#from imblearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report

In [5]:
# Trying to figure out SMOTE
from imblearn.over_sampling import SMOTE, SMOTENC
#df_train_downsample.head()

In [6]:

def train_NB_classifier(X_train, y_train, X_test):
    """
    Trains an Naive Bayes Multinominal model using a pipeline that includes a frequency encoding transformation,
    OneHotEncoder, and hyperparameter tuning via RandomizedSearchCV.
    
    Parameters:
        X_train (pd.DataFrame): Training features.
        y_train (pd.Series or np.array): Training target values.
        X_test (pd.DataFrame): Test features.
    
    Returns:
        best_estimator: The best estimator from RandomizedSearchCV.
        test_predictions: The predicted labels for X_test from the best estimator.
    """
    # Construct the pipeline:
    #   1. Apply frequency encoding (for example, on 'primary_color' & 'breed' if implemented in apply_freq_encode)
    #   2. OneHotEncode the features (adjust handle_unknown and sparse_output as needed)
    #   3. Fit an XGBClassifier.
    smote = SMOTENC(categorical_features=['intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'is_mix', 'primary_color'], random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

    pipeline = Pipeline([
        ('freq', FunctionTransformer(apply_freq_encode, validate=False)),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True)),
        ('nb', MultinomialNB())
    ])
    
    # Set up parameter distributions for XGBoost.
    param_distributions = {
        "nb__alpha": [0.1, 0.5, 1, 2, 5, 10],
    }
    
    # Perform hyperparameter search using RandomizedSearchCV.
    randomized_search = RandomizedSearchCV(
        estimator=pipeline, 
        param_distributions=param_distributions,
        n_iter=1,
        cv=5, 
        scoring='balanced_accuracy', 
        verbose=3,
    )
    
    randomized_search.fit(X_resampled, y_resampled)
    
    print('Best parameters:', randomized_search.best_params_)
    print('Best cross-validation accuracy:', randomized_search.best_score_)
    
    cv_scores = cross_val_score(randomized_search.best_estimator_, X_resampled, y_resampled, cv=5, verbose=3, scoring='balanced_accuracy')
    print('Generalization Balanced accuracy (via cross_val_score):', cv_scores.mean())

    # Make predictions on the test set using the best estimator.
    test_predictions = randomized_search.predict(X_test)

    
    
    return randomized_search.best_estimator_, test_predictions


In [7]:
# downsampling
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

df_train_downsample = resample(df_train, replace=False, n_samples=50000, random_state=42)
print(df_train_downsample.shape)

(50000, 10)


In [8]:
# Cross Validation
practice_dog = df_train#[df_train_downsample['animal_type'] == 'Dog']
practice_dog_labels = practice_dog['outcome_type']
practice_dog_data = practice_dog.drop(columns=['outcome_type'])
practice_train, practice_test, labels_train, labels_test = train_test_split(practice_dog_data,practice_dog_labels , test_size=0.30, random_state=42)

smote = SMOTENC(categorical_features=['intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'is_mix', 'primary_color'], random_state=42)
X_resampled, y_resampled = smote.fit_resample(practice_train, labels_train)

pipeline = Pipeline([
    ('freq', FunctionTransformer(apply_freq_encode, validate=False)),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True)),
   # ('smote', SMOTE(random_state=42)),  # <-- insert SMOTE here
    ('nb', MultinomialNB())
])

# Set up parameter distributions for XGBoost.
param_distributions = {
    "nb__alpha": [0.1,.2, .3, .4, 0.5, 1, 2, 5, 10],
}

# Perform hyperparameter search using RandomizedSearchCV.
randomized_search = RandomizedSearchCV(
    estimator=pipeline, 
    param_distributions=param_distributions,
    n_iter=1,
    cv=5, 
    scoring='balanced_accuracy', 
    verbose=3,
)

randomized_search.fit(X_resampled, y_resampled)

print('Best parameters:', randomized_search.best_params_)
print('Best estimates:', randomized_search.best_estimator_)
print('Best cross-validation accuracy:', randomized_search.best_score_)

cv_scores = cross_val_score(randomized_search.best_estimator_, X_resampled, y_resampled, cv=5, verbose=3, scoring='balanced_accuracy')
print('Generalization Balanced accuracy (via cross_val_score):', cv_scores.mean())

test_pred = randomized_search.predict(practice_test)

'''
NB_conf = confusion_matrix(y_true=labels_test, y_pred=test_pred)
print("Confusion Matrix:\n", NB_conf)

# Generate classification report
report = classification_report(y_true=labels_test, y_pred=test_pred)
print("Classification Report:\n", report)
'''


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/tmp/ipykernel_163125/2931022905.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(freq_series)
/tmp/ipykernel_163125/2931022905.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(freq_series)
/tmp/ipykernel_163125/2931022905.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/u

[CV 1/5] END .......................nb__alpha=5;, score=0.472 total time=   0.5s


/tmp/ipykernel_163125/2931022905.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(freq_series)
/tmp/ipykernel_163125/2931022905.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(freq_series)


[CV 2/5] END .......................nb__alpha=5;, score=0.511 total time=   0.5s


...

[CV] END ................................ score: (test=0.521) total time=   0.4s


/tmp/ipykernel_163125/2931022905.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(freq_series)
/tmp/ipykernel_163125/2931022905.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(freq_series)


[CV] END ................................ score: (test=0.494) total time=   0.4s
[CV] END ................................ score: (test=0.501) total time=   0.4s
Generalization Balanced accuracy (via cross_val_score): 0.500106701366298


/tmp/ipykernel_163125/2931022905.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(freq_series)
/tmp/ipykernel_163125/2931022905.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(freq_series)


'\nNB_conf = confusion_matrix(y_true=labels_test, y_pred=test_pred)\nprint("Confusion Matrix:\n", NB_conf)\n\n# Generate classification report\nreport = classification_report(y_true=labels_test, y_pred=test_pred)\nprint("Classification Report:\n", report)\n'

In [9]:
practice_dog.columns

Index(['intake_time', 'intake_type', 'intake_condition', 'animal_type',
       'sex_upon_intake', 'age_upon_intake', 'breed', 'is_mix',
       'primary_color', 'outcome_type'],
      dtype='object')

In [ ]:
# One Iteration of NB
practice_dog = df_train#[df_train_downsample['animal_type'] == 'Dog']
practice_dog_labels = practice_dog['outcome_type']
practice_dog_data = practice_dog.drop(columns=['outcome_type'])
practice_train, practice_test, labels_train, labels_test = train_test_split(practice_dog_data,practice_dog_labels , test_size=0.30, random_state=42)


smote = SMOTENC(categorical_features=['intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'is_mix', 'primary_color'], random_state=42)
X_resampled, y_resampled = smote.fit_resample(practice_train, labels_train)

ft =  FunctionTransformer(apply_freq_encode, validate=False)
X_freq = ft.fit_transform(X_resampled)

oh = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
X_encoded = oh.fit_transform(X_freq)

nb = MultinomialNB(alpha=10)
nb.fit(X_encoded, y_resampled)

X_test_freq = ft.transform(practice_test)
X_test_encoded = oh.transform(X_test_freq)

# Predict
y_pred = nb.predict(X_test_encoded)

# Evaluate
from sklearn.metrics import classification_report, confusion_matrix
classification_report_with_accuracy_score(labels_test, y_pred)

# Convert log probabilities to actual probabilities
feature_probs = np.exp(nb.feature_log_prob_)  # shape: (n_classes, n_features)


feature_names = oh.get_feature_names_out()  # after one-hot encoding


'''
pd.set_option('display.max_rows', None)       # Show all rows
pd.set_option('display.max_columns', None)    # Show all columns
pd.set_option('display.width', None)          # No line wrapping
#pd.set_option('display.max_colwidth', None)   # Don't truncate column content
'''



# Build DataFrame: rows = features, columns = classes
feature_prob_df = pd.DataFrame(feature_probs.T, index=feature_names, columns=nb.classes_)
cleaned_df = feature_prob_df[~feature_prob_df.index.str.contains("intake_time")]
no_breed_no_color = cleaned_df[~cleaned_df.index.str.contains("primary_color")]
no_breed_no_color = no_breed_no_color[~no_breed_no_color.index.str.contains("breed")]


/tmp/ipykernel_163125/2931022905.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(freq_series)


                 precision    recall  f1-score   support

       Adoption       0.63      0.79      0.70     16619
           Died       0.06      0.12      0.08       306
     Euthanasia       0.27      0.34      0.30      1035
Return to Owner       0.48      0.58      0.53      4827
       Transfer       0.69      0.31      0.43     10560

       accuracy                           0.59     33347
      macro avg       0.43      0.43      0.41     33347
   weighted avg       0.61      0.59      0.57     33347



,Adoption,Died,Euthanasia,Return to Owner,Transfer
intake_type_Abandoned,0.000177,0.000004,0.000014,0.000063,0.000110
intake_type_Euthanasia Request,0.000005,0.000004,0.000328,0.000005,0.000007
intake_type_Owner Surrender,0.003165,0.001398,0.001888,0.000749,0.002060
intake_type_Public Assist,0.000264,0.000015,0.000580,0.002954,0.000272
intake_type_Stray,0.008976,0.011164,0.009775,0.008814,0.010138
...,...,...,...,...,...
age_upon_intake_21.151482982566677,0.000003,0.000003,0.000003,0.000004,0.000003
age_upon_intake_22.0,0.000003,0.000003,0.000003,0.000004,0.000003
age_upon_intake_24.0,0.000003,0.000003,0.000003,0.000004,0.000003
is_mix_0,0.004947,0.005088,0.003541,0.004463,0.004356


In [17]:
no_breed_no_color.iloc[60:, :]

,Adoption,Died,Euthanasia,Return to Owner,Transfer
age_upon_intake_0.0002300549036006838,0.000003,0.000004,0.000003,0.000003,0.000003
age_upon_intake_0.0002373323443590971,0.000003,0.000004,0.000003,0.000003,0.000003
age_upon_intake_0.0002378207571990604,0.000003,0.000004,0.000003,0.000003,0.000003
age_upon_intake_0.00024073379689816099,0.000003,0.000004,0.000003,0.000003,0.000003
age_upon_intake_0.00024083506444665115,0.000003,0.000004,0.000003,0.000003,0.000003
...,...,...,...,...,...
age_upon_intake_21.151482982566677,0.000003,0.000003,0.000003,0.000004,0.000003
age_upon_intake_22.0,0.000003,0.000003,0.000003,0.000004,0.000003
age_upon_intake_24.0,0.000003,0.000003,0.000003,0.000004,0.000003
is_mix_0,0.004947,0.005088,0.003541,0.004463,0.004356


In [ ]:
from sklearn.preprocessing import LabelEncoder
# For Dog:
'''
train_dog = df_train_downsample[df_train_downsample['animal_type'] == 'Dog'].copy()
X_train_dog = train_dog.drop(columns=['animal_type', 'outcome_type'])
y_train_dog = train_dog['outcome_type']

test_dog = df_test[df_test['animal_type'] == 'Dog'].copy()
X_test_dog = test_dog.drop(columns=['animal_type'])

# For Cat:
train_cat = df_train_downsample[df_train_downsample['animal_type'] == 'Cat'].copy()
X_train_cat = train_cat.drop(columns=['animal_type', 'outcome_type'])
y_train_cat = train_cat['outcome_type']

test_cat = df_test[df_test['animal_type'] == 'Cat'].copy()
X_test_cat = test_cat.drop(columns=['animal_type'])

## Encode targets with LabelEncoder
# Dog encoding
le_dog = LabelEncoder()
y_train_dog_encoded = le_dog.fit_transform(y_train_dog)

# Cat encoding
le_cat = LabelEncoder()
y_train_cat_encoded = le_cat.fit_transform(y_train_cat)
'''

'''
print("Training model for Dog data:")
best_estimator_dog, dog_predictions_encoded = train_NB_classifier(X_train_dog, y_train_dog_encoded, X_test_dog)
dog_predictions = le_dog.inverse_transform(dog_predictions_encoded)

'''
# Don't split cat and dog
X_train = df_train.drop(columns=['outcome_type'])
y_train = df_train['outcome_type']

X_test = df_test

le_train = LabelEncoder()
y_train_encoded = le_train.fit_transform(y_train)


print("\nTraining model")
best_estimator, predictions_encoded = train_NB_classifier(X_train, y_train_encoded, X_test)
predictions = le_train.inverse_transform(predictions_encoded)


save_predictions(predictions, "NB")
print("DONE!\n", best_estimator)



Training model


KeyboardInterrupt: 